# TORONTO NEIGHBORHOODS SEGMENTATION AND CLUSTERING

## Importing Required Libraries

In [64]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [65]:
nbrhd=pd.read_csv('toronto_full.csv')
nbrhd.head()

,Unnamed: 0,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [66]:
df=nbrhd.drop('Unnamed: 0', axis=1) #Dropping the Unnamed: 0 column and creating the required dataframe
df.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [67]:
print('The dataset has {} number of Boroughs and {} number of Neighborhoods'.format(len(df['Borough'].unique()),len(df['Neighborhood'])))

The dataset has 10 number of Boroughs and 103 number of Neighborhods


## Importing Geospatial Data of Toronto

In [68]:
address = 'Toronto, Ontario'
geolocator = Nominatim(user_agent="tr_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


## Creating Map of Toronto City

In [69]:

Toronto_map = folium.Map(location=[latitude, longitude], zoom_start=10)

# Using the location information from toronto_full file, we add markers on the map for each neighbourhood

for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough) # This will label the markers by neighborhood, borough name
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        ).add_to(Toronto_map)  
    
Toronto_map

# Toronto Neighborhood Analysis

For this project, the borough to be evaluated will be Scarborough and its neighborhoods will be clustered.
In the code below, we will look for boroughs that contain "Scarborough".

In [70]:
# getting geospatial data of Scarborough
address = 'Scarborough, TO'
geolocator = Nominatim(user_agent="sb_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Scarborough are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Scarborough are 43.773077, -79.257774.


In [71]:
# retrieving data and fitting them into a new dataframe
sb_data=df[df['Borough'] == 'Scarborough'].reset_index(drop=True) 
sb_data.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [72]:
# creating map of Scarborough using latitude and longitude values
map_sb = folium.Map(location=[latitude, longitude], zoom_start=11)

# adding markers to map
for lat, lng, label in zip(sb_data['Latitude'], sb_data['Longitude'], sb_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_sb)  
    
map_sb

## Using FourSquare API to get venue details

In [97]:
CLIENT_ID = 0 #hidden 
CLIENT_SECRET = 0 #hidden
VERSION = '20200524'


In [74]:
sb_data.loc[0, 'Neighborhood'] #for the first entry in the dataframe

'Malvern, Rouge'

In [75]:
#getting co-ordinates of all the neighborhoods
print(*zip(sb_data['Latitude'], sb_data['Longitude'], sb_data['Neighborhood'])) 

(43.8066863, -79.19435340000003, 'Malvern, Rouge') (43.7845351, -79.16049709999999, 'Rouge Hill, Port Union, Highland Creek') (43.7635726, -79.1887115, 'Guildwood, Morningside, West Hill') (43.7709921, -79.21691740000001, 'Woburn') (43.773136, -79.23947609999998, 'Cedarbrae') (43.7447342, -79.23947609999998, 'Scarborough Village') (43.7279292, -79.26202940000002, 'Kennedy Park, Ionview, East Birchmount Park') (43.7111117, -79.2845772, 'Golden Mile, Clairlea, Oakridge') (43.716316, -79.23947609999998, 'Cliffside, Cliffcrest, Scarborough Village West') (43.692657, -79.2648481, 'Birch Cliff, Cliffside West') (43.7574096, -79.27330400000002, 'Dorset Park, Wexford Heights, Scarborough Town Centre') (43.7500715, -79.2958491, 'Wexford, Maryvale') (43.7942003, -79.26202940000002, 'Agincourt') (43.7816375, -79.3043021, "Clarks Corners, Tam O'Shanter, Sullivan") (43.8152522, -79.2845772, "Milliken, Agincourt North, Steeles East, L'Amoreaux East") (43.79952520000001, -79.3183887, "Steeles West, L

## Exploring Venues
Now we create a funtion that extracts name, latitude and longitude from our dataset. Using this information and our FourSquare API credentials, it generates output of venue data around 500m radius of the neighborhood in the form of json code. Then, the relevant information is used to create a dataframe.

In [76]:

def NearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [77]:
LIMIT = 100
radius = 500

sb_venues = NearbyVenues(names=sb_data['Neighborhood'],
                                   latitudes=sb_data['Latitude'],
                                   longitudes=sb_data['Longitude']
                                  )

Malvern, Rouge
Rouge Hill, Port Union, Highland Creek
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
Kennedy Park, Ionview, East Birchmount Park
Golden Mile, Clairlea, Oakridge
Cliffside, Cliffcrest, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Wexford Heights, Scarborough Town Centre
Wexford, Maryvale
Agincourt
Clarks Corners, Tam O'Shanter, Sullivan
Milliken, Agincourt North, Steeles East, L'Amoreaux East
Steeles West, L'Amoreaux West
Upper Rouge


In [78]:
print(sb_venues.shape)
sb_venues.head()

(91, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Malvern, Rouge",43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Chris Effects Painting,43.784343,-79.163742,Construction & Landscaping
2,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
3,"Guildwood, Morningside, West Hill",43.763573,-79.188711,RBC Royal Bank,43.766790,-79.191151,Bank
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store


In [79]:
#number of venues returned per neighborhood
sb_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Agincourt,4,4,4,4,4,4
"Birch Cliff, Cliffside West",4,4,4,4,4,4
Cedarbrae,9,9,9,9,9,9
"Clarks Corners, Tam O'Shanter, Sullivan",14,14,14,14,14,14
"Cliffside, Cliffcrest, Scarborough Village West",2,2,2,2,2,2
"Dorset Park, Wexford Heights, Scarborough Town Centre",7,7,7,7,7,7
"Golden Mile, Clairlea, Oakridge",10,10,10,10,10,10
"Guildwood, Morningside, West Hill",7,7,7,7,7,7
"Kennedy Park, Ionview, East Birchmount Park",5,5,5,5,5,5


In [98]:
#number of venues returned per neighborhood
sb_venues.groupby('Venue Category').count()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude
Venue Category,,,,,,
American Restaurant,1,1,1,1,1,1
Athletics & Sports,1,1,1,1,1,1
Auto Garage,1,1,1,1,1,1
Bakery,4,4,4,4,4,4
Bank,4,4,4,4,4,4
Bar,1,1,1,1,1,1
Breakfast Spot,4,4,4,4,4,4
Bus Line,2,2,2,2,2,2
Bus Station,2,2,2,2,2,2


In [80]:
print('There are {} uniques categories.'.format(len(sb_venues['Venue Category'].unique())))

There are 52 uniques categories.


## Frequency of occurence of each category in a neighborhood
For this, we will use one-hot encoding. 

In [81]:
# one hot encoding
sb_onehot = pd.get_dummies(sb_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
sb_onehot['Neighborhood'] = sb_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [sb_onehot.columns[-1]] + list(sb_onehot.columns[:-1])
sb_onehot = sb_onehot[fixed_columns]

sb_onehot.head()

,Neighborhood,American Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Bus Line,Bus Station,Café,Caribbean Restaurant,Chinese Restaurant,Coffee Shop,College Stadium,Construction & Landscaping,Convenience Store,Department Store,Discount Store,Electronics Store,Fast Food Restaurant,Fried Chicken Joint,Gas Station,General Entertainment,Grocery Store,Hakka Restaurant,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Korean Restaurant,Latin American Restaurant,Light Rail Station,Lounge,Medical Center,Metro Station,Mexican Restaurant,Motel,Nail Salon,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Rental Car Location,Sandwich Place,Shopping Mall,Skating Rink,Soccer Field,Thai Restaurant,Thrift / Vintage Store,Vietnamese Restaurant
0,"Malvern, Rouge",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Rouge Hill, Port Union, Highland Creek",0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Rouge Hill, Port Union, Highland Creek",0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"Guildwood, Morningside, West Hill",0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [82]:
sb_onehot.shape

(91, 53)

Now, we find the mean frequency of occurence of each venue in a neighborhood.

In [83]:
sb_grouped = sb_onehot.groupby('Neighborhood').mean().reset_index()
sb_grouped.head()

,Neighborhood,American Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Bus Line,Bus Station,Café,Caribbean Restaurant,Chinese Restaurant,Coffee Shop,College Stadium,Construction & Landscaping,Convenience Store,Department Store,Discount Store,Electronics Store,Fast Food Restaurant,Fried Chicken Joint,Gas Station,General Entertainment,Grocery Store,Hakka Restaurant,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Korean Restaurant,Latin American Restaurant,Light Rail Station,Lounge,Medical Center,Metro Station,Mexican Restaurant,Motel,Nail Salon,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Rental Car Location,Sandwich Place,Shopping Mall,Skating Rink,Soccer Field,Thai Restaurant,Thrift / Vintage Store,Vietnamese Restaurant
0,Agincourt,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.25,0.0,0.0,0.00,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.25,0.0,0.250000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.25,0.0,0.000000,0.0,0.0
1,"Birch Cliff, Cliffside West",0.0,0.000000,0.0,0.000000,0.000000,0.0,0.00,0.0,0.0,0.25,0.000000,0.000000,0.000000,0.25,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.25,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.00,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.25,0.0,0.000000,0.0,0.0
2,Cedarbrae,0.0,0.111111,0.0,0.111111,0.111111,0.0,0.00,0.0,0.0,0.00,0.111111,0.000000,0.000000,0.00,0.0,0.000000,0.0,0.0,0.0,0.000000,0.111111,0.111111,0.00,0.0,0.111111,0.0,0.0,0.0,0.000000,0.0,0.00,0.0,0.111111,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.00,0.0,0.111111,0.0,0.0
3,"Clarks Corners, Tam O'Shanter, Sullivan",0.0,0.000000,0.0,0.000000,0.071429,0.0,0.00,0.0,0.0,0.00,0.000000,0.071429,0.071429,0.00,0.0,0.071429,0.0,0.0,0.0,0.071429,0.071429,0.071429,0.00,0.0,0.000000,0.0,0.0,0.0,0.071429,0.0,0.00,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.071429,0.0,0.0,0.142857,0.142857,0.0,0.0,0.0,0.0,0.00,0.0,0.071429,0.0,0.0
4,"Cliffside, Cliffcrest, Scarborough Village West",0.5,0.000000,0.0,0.000000,0.000000,0.0,0.00,0.0,0.0,0.00,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.00,0.0,0.000000,0.0,0.0,0.0,0.5,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.00,0.0,0.000000,0.0,0.0


In [84]:
sb_grouped.shape

(16, 53)

## Finding top 5 venues in a neighborhood

In [40]:
num_top_venues = 5

for hood in sb_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = sb_grouped[sb_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0  Latin American Restaurant  0.25
1               Skating Rink  0.25
2             Breakfast Spot  0.25
3                     Lounge  0.25
4        American Restaurant  0.00


----Birch Cliff, Cliffside West----
                   venue  freq
0        College Stadium  0.25
1           Skating Rink  0.25
2  General Entertainment  0.25
3                   Café  0.25
4                   Park  0.00


----Cedarbrae----
                venue  freq
0              Lounge  0.11
1              Bakery  0.11
2                Bank  0.11
3     Thai Restaurant  0.11
4  Athletics & Sports  0.11


----Clarks Corners, Tam O'Shanter, Sullivan----
                 venue  freq
0             Pharmacy  0.14
1          Pizza Place  0.14
2    Convenience Store  0.07
3         Noodle House  0.07
4  Fried Chicken Joint  0.07


----Cliffside, Cliffcrest, Scarborough Village West----
                 venue  freq
0  American Restaurant   0.5
1                Mot

## Finding 10 most common venues in a neighborhood and fitting them into a dataframe

In [85]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [86]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# Labelling columns as 1st, 2nd and so on
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind])) # for 1st, 2nd, 3rd
    except:
        columns.append('{}th Most Common Venue'.format(ind+1)) ### for 4th, 5th,...

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns) ## assign column names we just created to a new dataframe
neighborhoods_venues_sorted['Neighborhood'] = sb_grouped['Neighborhood']## add neighborhoods column

for ind in np.arange(sb_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(sb_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Skating Rink,Latin American Restaurant,Breakfast Spot,Lounge,Vietnamese Restaurant,College Stadium,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
1,"Birch Cliff, Cliffside West",General Entertainment,Skating Rink,Café,College Stadium,Vietnamese Restaurant,Coffee Shop,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
2,Cedarbrae,Thai Restaurant,Athletics & Sports,Bakery,Bank,Gas Station,Fried Chicken Joint,Caribbean Restaurant,Hakka Restaurant,Lounge,Vietnamese Restaurant
3,"Clarks Corners, Tam O'Shanter, Sullivan",Pharmacy,Pizza Place,Gas Station,Italian Restaurant,Noodle House,Convenience Store,Coffee Shop,Chinese Restaurant,Bank,Fast Food Restaurant
4,"Cliffside, Cliffcrest, Scarborough Village West",American Restaurant,Motel,Hakka Restaurant,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store


## Clustering the neighborhoods

In [87]:
# set number of clusters
kclusters = 5

sb_grouped_clustering = sb_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(sb_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 0, 1, 1, 1, 1, 4])

In [88]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

sb_merged = sb_data

# merge sb_grouped with sb_data to add latitude/longitude for each neighborhood
sb_merged = sb_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

sb_merged

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,4.0,Fast Food Restaurant,Vietnamese Restaurant,Thrift / Vintage Store,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Electronics Store,Discount Store,Department Store
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,1.0,Bar,Construction & Landscaping,Vietnamese Restaurant,College Stadium,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,1.0,Mexican Restaurant,Intersection,Bank,Rental Car Location,Breakfast Spot,Electronics Store,Medical Center,Vietnamese Restaurant,Gas Station,Fried Chicken Joint
3,M1G,Scarborough,Woburn,43.770992,-79.216917,3.0,Coffee Shop,Indian Restaurant,Korean Restaurant,College Stadium,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,1.0,Thai Restaurant,Athletics & Sports,Bakery,Bank,Gas Station,Fried Chicken Joint,Caribbean Restaurant,Hakka Restaurant,Lounge,Vietnamese Restaurant
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476,2.0,Playground,Vietnamese Restaurant,Coffee Shop,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029,1.0,Coffee Shop,Discount Store,Bus Station,Department Store,Convenience Store,College Stadium,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577,1.0,Bakery,Bus Line,Ice Cream Shop,Metro Station,Soccer Field,Intersection,Bus Station,Park,Gas Station,Fried Chicken Joint
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476,0.0,American Restaurant,Motel,Hakka Restaurant,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848,1.0,General Entertainment,Skating Rink,Café,College Stadium,Vietnamese Restaurant,Coffee Shop,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store


In [89]:
sb_merged.dropna(inplace=True) #dropping rows with no value

## Creating a map of clusters

In [90]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]


# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(sb_merged['Latitude'], sb_merged['Longitude'], sb_merged['Neighborhood'], sb_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examining the clusters

In [91]:
# Red Marker
sb_merged.loc[sb_merged['Cluster Labels'] == 0, sb_merged.columns[[1] + list(range(5, sb_merged.shape[1]))]] 

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Scarborough,0.0,American Restaurant,Motel,Hakka Restaurant,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store


In [92]:
# Purple Marker
sb_merged.loc[sb_merged['Cluster Labels'] == 1, sb_merged.columns[[1] + list(range(5, sb_merged.shape[1]))]] 

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Scarborough,1.0,Bar,Construction & Landscaping,Vietnamese Restaurant,College Stadium,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
2,Scarborough,1.0,Mexican Restaurant,Intersection,Bank,Rental Car Location,Breakfast Spot,Electronics Store,Medical Center,Vietnamese Restaurant,Gas Station,Fried Chicken Joint
4,Scarborough,1.0,Thai Restaurant,Athletics & Sports,Bakery,Bank,Gas Station,Fried Chicken Joint,Caribbean Restaurant,Hakka Restaurant,Lounge,Vietnamese Restaurant
6,Scarborough,1.0,Coffee Shop,Discount Store,Bus Station,Department Store,Convenience Store,College Stadium,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint
7,Scarborough,1.0,Bakery,Bus Line,Ice Cream Shop,Metro Station,Soccer Field,Intersection,Bus Station,Park,Gas Station,Fried Chicken Joint
9,Scarborough,1.0,General Entertainment,Skating Rink,Café,College Stadium,Vietnamese Restaurant,Coffee Shop,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
10,Scarborough,1.0,Indian Restaurant,Vietnamese Restaurant,Thrift / Vintage Store,Light Rail Station,Pet Store,Chinese Restaurant,College Stadium,Gas Station,Fried Chicken Joint,Fast Food Restaurant
11,Scarborough,1.0,Auto Garage,Bakery,Shopping Mall,Sandwich Place,Breakfast Spot,Vietnamese Restaurant,College Stadium,General Entertainment,Gas Station,Fried Chicken Joint
12,Scarborough,1.0,Skating Rink,Latin American Restaurant,Breakfast Spot,Lounge,Vietnamese Restaurant,College Stadium,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
13,Scarborough,1.0,Pharmacy,Pizza Place,Gas Station,Italian Restaurant,Noodle House,Convenience Store,Coffee Shop,Chinese Restaurant,Bank,Fast Food Restaurant


In [93]:
# Blue Marker
sb_merged.loc[sb_merged['Cluster Labels'] == 2, sb_merged.columns[[1] + list(range(5, sb_merged.shape[1]))]] 

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Scarborough,2.0,Playground,Vietnamese Restaurant,Coffee Shop,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store
14,Scarborough,2.0,Playground,Park,Vietnamese Restaurant,Coffee Shop,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store


In [94]:
# Green Marker
sb_merged.loc[sb_merged['Cluster Labels'] == 3, sb_merged.columns[[1] + list(range(5, sb_merged.shape[1]))]] 

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Scarborough,3.0,Coffee Shop,Indian Restaurant,Korean Restaurant,College Stadium,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store


In [95]:
# Yellow Marker
sb_merged.loc[sb_merged['Cluster Labels'] == 4, sb_merged.columns[[1] + list(range(5, sb_merged.shape[1]))]] 

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,4.0,Fast Food Restaurant,Vietnamese Restaurant,Thrift / Vintage Store,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Electronics Store,Discount Store,Department Store
